In [1]:
import importlib
import sys,os,math,time

import networkx as nx
import geopandas as gpd
import numpy as np
import osmnx as ox
import pandas as pd
import rasterio
import pylab as plt
import json
import datetime
import pycrs
import seaborn as sns
from glob import glob
from rasterio.plot import show, show_hist
from rasterio.mask import mask
from shapely.geometry import Point
from shapely.geometry import LineString
sys.path.append('GOSTNets')

import GOSTnet as gn
from pandana.loaders import osm;

from osgeo import gdal, gdalconst, gdal_array
from rasterstats import point_query, zonal_stats
from make_fishnet import make_fishnet

peartree version: 0.6.1 
networkx version: 2.2 
matplotlib version: 3.0.2 
osmnx version: 0.9 


## Functions

In [2]:
def get_amenities(string, tag):
    
    if tag == 'healthcare':
        amenities = ['hospital', 'doctor', 'healthcare']
    elif tag=='schools':
        amenities = ['school', 'university', 'college']
    else:
        print('Invalid tag. Enter either healthcare or schools.')


    osm_tags = '"amenity"~"{}"'.format('|'.join(amenities))

    #pois = osm.node_query(coords[0],coords[1],coords[2],coords[3],tags=osm_tags) # required: lat_min, lng_min, lat_max, lng_max
    
    poly = get_polygon(string) ### minx	miny	maxx	maxy
    
    extent = poly.bounds

    pois = osm.node_query(extent[1], extent[0], extent[3], extent[2], tags=osm_tags)

    pois = pois[pois['amenity'].isin(amenities)]

    try:
        osm_data = pois[['amenity', 'lat', 'lon', 'name', 'name:en']]
    except:
        osm_data = pois[['amenity', 'lat', 'lon', 'name']]


    osm_data['geometry'] = (list(zip(osm_data.lon,osm_data.lat)))
    osm_data['geometry'] = osm_data.geometry.apply(lambda x: Point(x))
    osm_data = gpd.GeoDataFrame(osm_data, geometry='geometry')
    
    osm_data.crs = {'init':'epsg:4326'}
    string = str(string)
    print('Saving OSM data to shapefile at path data/{}'.format(string))
    #osm_data.to_file('data/{}/{}_{}.shp'.format(string, string, tag))
    
    return osm_data

In [3]:
def get_polygon(string):
    
    lis = glob('/nfshome/gb1877/gb1877/WorldBank/CRP/Betweenness_Centrality/data/AOI/*/*.shp')
    
    city = []
    for i in lis:
        x = i.split('/')[-2]
        city.append(x.split('_')[0])
        
    if string in city:
        l = glob('/nfshome/gb1877/gb1877/WorldBank/CRP/Betweenness_Centrality/data/AOI/{}_AOI/*.shp'.format(string))
        adm = gpd.GeoDataFrame.from_file(l[0])
        boundary_poly = adm.geometry.values[0]
    else:
        boundary_GDF = ox.gdf_from_place('{}'.format(string),which_result=1)
        boundary_poly = boundary_GDF.loc[0,'geometry']
        if boundary_poly.geom_type == 'Polygon':
            boundary_poly = boundary_poly
        else:
            try:
                boundary_GDF = ox.gdf_from_place('{}'.format(string),which_result=2)
                boundary_poly = boundary_GDF.loc[0,'geometry']
            except:
                print('Polygon not available')
    
    return boundary_poly

In [8]:
def get_graph(string, boundary_GDF, network_type):
    
    print('Fetching graph data for {}'.format(string))

    G = ox.graph_from_polygon(get_polygon(string), network_type = network_type)
    
    node_dict = {}

    G2 = G.copy()

    for u, data in G2.nodes(data = True):
        node_dict[u] = Point(data['x'],data['y'])

    for u, v, data in G2.edges(data = True):

        if 'Wkt' not in data.keys():
            strt = node_dict[u]
            fini = node_dict[v]
            data['Wkt'] = LineString([strt, fini])

        if 'geometry' in data.keys():
            data['Wkt'] = data['geometry']
            data.pop('geometry')
            
    importlib.reload(gn)
    G3 = gn.salt_long_lines(G2,
                           source = 'epsg:4326',
                          target = 'epsg:3857',
                          thresh = 50,
                          factor = 1)
    
    return G3
    

In [5]:
def get_time(t1, t2):
    
    diff = t2 - t1
    
    c = round(diff.total_seconds() / 60, 2)
    
    return c

In [31]:
def get_accessibility(place, grid_size=100):
        
    t1= datetime.datetime.now()
    string = place.split(',')[0]
    
    boundary_GDF = ox.gdf_from_place('{}'.format(string),which_result=1)
    
    osm_schools = get_amenities(string, 'schools')
    
    osm_hosp = get_amenities(string, 'healthcare')

    t2 = datetime.datetime.now()
    
    print('{} minutes elapsed!'.format(get_time(t1, t2)))
    #### For drive network
    
    G_drive = get_graph(string,boundary_GDF, 'drive')
    
    t3 = datetime.datetime.now()
    
    print('{} minutes elapsed!'.format(get_time(t1, t3)))
    print('Snapping OSM data to drive network')
    
    schools = gn.pandana_snap(G_drive, osm_schools)
    
    health = gn.pandana_snap(G_drive, osm_hosp)
    
    #node_gdf_drive = ox.graph_to_gdfs(G_drive, nodes=True, edges=False)
    #node_gdf_drive = gn.pandana_snap(G_drive, node_gdf_drive)
    
    t4 = datetime.datetime.now()
    
    print('{} minutes elapsed!'.format(get_time(t1, t4)))
    
    fail_value = 99999999
    
    print('Creating fishnet')
    make_fishnet('data/{a}/{b}_{c}_grid.shp'.format(a=string,b=string, c='fishnet'),boundary_GDF.bbox_west[0],\
             boundary_GDF.bbox_east[0], boundary_GDF.bbox_south[0],  boundary_GDF.bbox_north[0],grid_size,grid_size) # takes xmin, xmax, ymin, ymax   
    
    
    grid=gpd.read_file('data/{a}/{b}_{c}_grid.shp'.format(a=string,b=string, c='fishnet'))
    
    grid['centroid'] = grid.geometry.centroid
    grid_graph = grid[['FID', 'centroid']]
    grid_graph = gpd.GeoDataFrame(grid_graph, geometry = 'centroid')
    grid_graph['geometry'] = grid_graph.centroid
    
    grid_graph = gn.pandana_snap(G_drive, grid_graph)
    
    t5 = datetime.datetime.now()
    
    print('{} minutes elapsed!'.format(get_time(t1, t5)))
    
    importlib.reload(gn)
    print('Calculating OD matrix for drive network')
    #schools_OD_drive = gn.calculate_OD(G=G_drive, origins=list(node_gdf_drive.NN), destinations = list(schools.NN), 
    #                             fail_value = fail_value, weight='time')
    
    #health_OD_drive = gn.calculate_OD(G=G_drive, origins=list(node_gdf_drive.NN), destinations = list(health.NN), 
    #                         fail_value = fail_value, weight='time')
    
    schools_OD_drive = gn.calculate_OD(G=G_drive, origins=list(grid_graph.NN), destinations = list(schools.NN), 
                                 fail_value = fail_value, weight='time')
    
    health_OD_drive = gn.calculate_OD(G=G_drive, origins=list(grid_graph.NN), destinations = list(health.NN), 
                             fail_value = fail_value, weight='time')
    
    #node_gdf_drive['distance_school'] = [schools_OD_drive[i].min() for i in range(schools_OD_drive.shape[0])]
    #node_gdf_drive['distance_hospital'] = [health_OD_drive[i].min() for i in range(health_OD_drive.shape[0])]
    
    grid['dis_dr_schl'] = [schools_OD_drive[i].min() for i in range(schools_OD_drive.shape[0])]
    grid['dis_dr_hosp'] = [health_OD_drive[i].min() for i in range(health_OD_drive.shape[0])]
    
    t6 = datetime.datetime.now()
    
    print('{} minutes elapsed!'.format(get_time(t1, t6)))
    ##### For walk network
    
    G_walk = get_graph(string,boundary_GDF, 'walk')
    
    print('Snapping OSM data to walk network')
    schools_walk = gn.pandana_snap(G_walk, osm_schools)
    
    health_walk = gn.pandana_snap(G_walk, osm_hosp)
    
    grid_graph_walk = gn.pandana_snap(G_walk, grid_graph)
    #node_gdf_walk = ox.graph_to_gdfs(G_walk, nodes=True, edges=False)
    #node_gdf_walk = gn.pandana_snap(G_walk, node_gdf_walk)
    
    t7 = datetime.datetime.now()
    
    print('{} minutes elapsed!'.format(get_time(t1, t7)))
    print('Calculating OD matrix for walk network')
    
    #schools_OD_walk = gn.calculate_OD(G=G_walk, origins=list(node_gdf_walk.NN), destinations = list(schools_walk.NN), 
    #                             fail_value = fail_value, weight='time')
    
    #health_OD_walk = gn.calculate_OD(G=G_walk, origins=list(node_gdf_walk.NN), destinations = list(health_walk.NN), 
    #                         fail_value = fail_value, weight='time')
    
    schools_OD_walk = gn.calculate_OD(G=G_walk, origins=list(grid_graph_walk.NN), destinations = list(schools_walk.NN), 
                                 fail_value = fail_value, weight='time')
    
    health_OD_walk = gn.calculate_OD(G=G_walk, origins=list(grid_graph_walk.NN), destinations = list(health_walk.NN), 
                             fail_value = fail_value, weight='time')
    
    #node_gdf_walk['distance_school'] = [schools_OD_walk[i].min() for i in range(schools_OD_walk.shape[0])]
    #node_gdf_walk['distance_hospital'] = [health_OD_walk[i].min() for i in range(health_OD_walk.shape[0])]
    
    grid['dis_wa_schl'] = [schools_OD_walk[i].min() for i in range(schools_OD_walk.shape[0])]
    grid['dis_wa_hosp'] = [health_OD_walk[i].min() for i in range(health_OD_walk.shape[0])]
    
    t8 = datetime.datetime.now()
    
    print('{} minutes elapsed!'.format(get_time(t1, t8)))
    
    #print('Snapping grid to network')
    #grid_graph_drive =  gn.pandana_snap(G_drive, grid_graph)
    #grid_graph_walk =  gn.pandana_snap(G_walk, grid_graph)
    
    #dis_school_drive = []
    #for i in range(grid_graph_drive.shape[0]):
    #    node = grid_graph_drive.NN[i]
    #    try:
    #        distance = node_gdf_drive.loc[node,'distance_school']
    #    except:
    #        pass
    #   dis_school_drive.append(distance)
        
    #dis_hosp_drive = []
    #for i in range(grid_graph_drive.shape[0]):
    #    node = grid_graph_drive.NN[i]
    #    try:
    #        distance = node_gdf_drive.loc[node,'distance_hospital']
    #    except:
    #        pass
    #    dis_hosp_drive.append(distance)
    
    #dis_school_walk = []
    #for i in range(grid_graph_walk.shape[0]):
    #    node = grid_graph_walk.NN[i]
    #    try:
    #        distance = node_gdf_walk.loc[node,'distance_school']
    #    except:
    #        pass
    #    dis_school_walk.append(distance)
        
    #dis_hosp_walk = []
    #for i in range(grid_graph_walk.shape[0]):
    #    node = grid_graph_walk.NN[i]
    #    try:
    #        distance = node_gdf_walk.loc[node,'distance_hospital']
    #    except:
    #        pass
    #    dis_hosp_walk.append(distance)
    
    #t9 = datetime.datetime.now()
    
    #print('{} minutes elapsed!'.format(get_time(t1, t9)))
    
    #grid['drive_school'] =  dis_school_drive
    #grid['drive_hosp'] = dis_hosp_drive
    #grid['walk_school'] = dis_school_walk
    #grid['walk_hosp'] = dis_hosp_walk
    
    grid['total_school'] = grid.dis_dr_schl + grid.dis_wa_schl
    grid['total_hosp'] = grid.dis_dr_hosp + grid.dis_wa_hosp
    
    grid.drop(columns=['centroid'], inplace=True)
    #grid.crs = {'init':'epsg:4326'}
    print('Writing output file!!')
    grid.to_file('data/{}/Access_School_Hosp_{}_{}.shp'.format(string, string, grid_size))
    print('Done!!!')

In [19]:
get_accessibility('Tashkent, Uzbekistan')

/nfshome/gb1877/.conda/envs/osm2/lib/python3.6/site-packages/ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/nfshome/gb1877/.conda/envs/osm2/lib/python3.6/site-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


0.06 minutes elapsed!
Fetching graph data for Tashkent
peartree version: 0.6.1 
networkx version: 2.2 
matplotlib version: 3.0.2 
osmnx version: 0.9 
Identified 17539 unique edge(s) longer than 50. 
Beginning new node creation...
152200 new edges added and 30568 removed to bring total edges to 158832
58561 new nodes added to bring total nodes to 73338
2.03 minutes elapsed!
Snapping OSM data to drive network
2.4 minutes elapsed!
peartree version: 0.6.1 
networkx version: 2.2 
matplotlib version: 3.0.2 
osmnx version: 0.9 
Calculating OD matrix for drive network
print origins type
<class 'list'>
weighted_origins equals false
print origins type
<class 'list'>
weighted_origins equals false
4.13 minutes elapsed!
Fetching graph data for Tashkent
peartree version: 0.6.1 
networkx version: 2.2 
matplotlib version: 3.0.2 
osmnx version: 0.9 
Identified 35618 unique edge(s) longer than 50. 
Beginning new node creation...
250618 new edges added and 71388 removed to bring total edges to 304190
896

/nfshome/gb1877/.conda/envs/osm2/lib/python3.6/site-packages/geopandas/io/file.py:108: FionaDeprecationWarning: Use fiona.Env() instead.
  with fiona.drivers():


In [20]:
shp = gpd.GeoDataFrame.from_file('data/Tashkent/Access_School_Hosp_Tashkent.shp')

In [21]:
shp.head()

,FID,drive_scho,drive_hosp,walk_schoo,walk_hosp,total_scho,total_hosp,geometry
0,0,4.0,26.0,0.0,31.0,4.0,57.0,"POLYGON ((69.1187079 41.4723363, 69.1196062152..."
1,1,4.0,26.0,0.0,31.0,4.0,57.0,"POLYGON ((69.1187079 41.47114325896642, 69.119..."
2,2,0.0,30.0,0.0,31.0,0.0,61.0,"POLYGON ((69.1187079 41.46995021793284, 69.119..."
3,3,2.0,32.0,0.0,31.0,2.0,63.0,"POLYGON ((69.1187079 41.46875717689926, 69.119..."
4,4,3.0,33.0,0.0,31.0,3.0,64.0,"POLYGON ((69.1187079 41.46756413586568, 69.119..."


In [37]:
shp.total_scho.max()

520.0

In [39]:
priority_list = [
    'Coron, Philippines', 'Puerto Princesa, Philippines', 'El Nido, Philippines', 'Davao City, Philippines', 
    'Tashkent, Uzbekistan' 'Hebron, West Bank', 'Bethlehem, West Bank', 'Lima, Peru', 'Vinh Long, Vietnam', 
    'Khanh Hoa, Vietnam', 'Osh, Kyrgyzstan', 'Monrovia, Liberia'
]

In [ ]:
for i in priority_list:
    %time get_accessibility(i)

Saving OSM data to shapefile at path data/Coron


/nfshome/gb1877/.conda/envs/osm2/lib/python3.6/site-packages/geopandas/io/file.py:108: FionaDeprecationWarning: Use fiona.Env() instead.
  with fiona.drivers():


Saving OSM data to shapefile at path data/Coron


ValueError: Cannot write empty DataFrame to file.

/nfshome/gb1877/.conda/envs/osm2/lib/python3.6/site-packages/ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/nfshome/gb1877/.conda/envs/osm2/lib/python3.6/site-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/nfshome/gb1877/.conda/envs/osm2/lib/python3.6/site-packages/geopandas/io/file.py:108: FionaDeprecationWarning: Use fiona.Env() instead.
  with fiona.drivers():


...

CPU times: user 3min 40s, sys: 2.04 s, total: 3min 42s
Wall time: 6min 43s
Saving OSM data to shapefile at path data/Lima


/nfshome/gb1877/.conda/envs/osm2/lib/python3.6/site-packages/geopandas/io/file.py:108: FionaDeprecationWarning: Use fiona.Env() instead.
  with fiona.drivers():


Saving OSM data to shapefile at path data/Lima
0.24 minutes elapsed!
Fetching graph data for Lima


/nfshome/gb1877/.conda/envs/osm2/lib/python3.6/site-packages/geopandas/io/file.py:108: FionaDeprecationWarning: Use fiona.Env() instead.
  with fiona.drivers():


peartree version: 0.6.1 
networkx version: 2.2 
matplotlib version: 3.0.2 
osmnx version: 0.9 
Identified 177764 unique edge(s) longer than 50. 
Beginning new node creation...


In [ ]:
%time get_accessibility('Lima, Peru')

/nfshome/gb1877/.conda/envs/osm2/lib/python3.6/site-packages/ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/nfshome/gb1877/.conda/envs/osm2/lib/python3.6/site-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Saving OSM data to shapefile at path data/Lima
Saving OSM data to shapefile at path data/Lima
0.06 minutes elapsed!
Fetching graph data for Lima
peartree version: 0.6.1 
networkx version: 2.2 
matplotlib version: 3.0.2 
osmnx version: 0.9 


## Testing

In [32]:
%time get_accessibility('Tashkent, Uzbekistan', 500)

/nfshome/gb1877/.conda/envs/osm2/lib/python3.6/site-packages/ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/nfshome/gb1877/.conda/envs/osm2/lib/python3.6/site-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Saving OSM data to shapefile at path data/Tashkent
Saving OSM data to shapefile at path data/Tashkent
0.18 minutes elapsed!
Fetching graph data for Tashkent
peartree version: 0.6.1 
networkx version: 2.2 
matplotlib version: 3.0.2 
osmnx version: 0.9 
Identified 18113 unique edge(s) longer than 50. 
Beginning new node creation...
180274 new edges added and 31528 removed to bring total edges to 185946
72024 new nodes added to bring total nodes to 86801
1.75 minutes elapsed!
Snapping OSM data to drive network
1.81 minutes elapsed!
Creating fishnet
1.89 minutes elapsed!
peartree version: 0.6.1 
networkx version: 2.2 
matplotlib version: 3.0.2 
osmnx version: 0.9 
Calculating OD matrix for drive network
print origins type
<class 'list'>
weighted_origins equals false
print origins type
<class 'list'>
weighted_origins equals false
3.73 minutes elapsed!
Fetching graph data for Tashkent
peartree version: 0.6.1 
networkx version: 2.2 
matplotlib version: 3.0.2 
osmnx version: 0.9 
Identified 38

/nfshome/gb1877/.conda/envs/osm2/lib/python3.6/site-packages/geopandas/io/file.py:108: FionaDeprecationWarning: Use fiona.Env() instead.
  with fiona.drivers():


Done!!!
CPU times: user 11min 13s, sys: 9.93 s, total: 11min 23s
Wall time: 28min 58s


In [13]:
boundary_GDF = ox.gdf_from_place('{}'.format('Tashkent, Uzbekistan'),which_result=1)

In [14]:
make_fishnet('data/{a}/{b}_{c}_grid.shp'.format(a='Tashkent',b='Tashkent', c='fishnet'),boundary_GDF.bbox_west[0],\
             boundary_GDF.bbox_east[0], boundary_GDF.bbox_south[0],  boundary_GDF.bbox_north[0],100,100) # takes xmin, xmax, ymin, ymax   

In [16]:
grid=gpd.read_file('data/{a}/{b}_{c}_grid.shp'.format(a='Tashkent',b='Tashkent', c='fishnet'))
    
grid['centroid'] = grid.geometry.centroid
grid_graph = grid[['FID', 'centroid']]
grid_graph = gpd.GeoDataFrame(grid_graph, geometry = 'centroid')

grid_graph['geometry'] = grid_graph.centroid

In [18]:
 G_drive = get_graph('Tashkent',boundary_GDF, 'drive')

Fetching graph data for Tashkent
peartree version: 0.6.1 
networkx version: 2.2 
matplotlib version: 3.0.2 
osmnx version: 0.9 
Identified 18113 unique edge(s) longer than 50. 
Beginning new node creation...
180274 new edges added and 31528 removed to bring total edges to 185946
72024 new nodes added to bring total nodes to 86801


In [21]:
test = gn.pandana_snap(G_drive, grid_graph)

In [20]:
grid_graph = grid[['FID', 'centroid']]
grid_graph = gpd.GeoDataFrame(grid_graph, geometry = 'centroid')
grid_graph['geometry'] = grid_graph.centroid

In [22]:
test.head()

,FID,centroid,geometry,x,y,NN
0,0,POINT (69.11915705764207 41.47173977948322),POINT (69.11915705764207 41.47173977948322),69.119157,41.471740,4310279974_7053_34316
1,1,POINT (69.11915705764207 41.47054673844963),POINT (69.11915705764207 41.47054673844963),69.119157,41.470547,4310279974_7057_34320
2,2,POINT (69.11915705764207 41.46935369741605),POINT (69.11915705764207 41.46935369741605),69.119157,41.469354,4310279974_7059_34322
3,3,POINT (69.11915705764207 41.46816065638247),POINT (69.11915705764207 41.46816065638247),69.119157,41.468161,4310279974_7060_34323
4,4,POINT (69.11915705764207 41.46696761534889),POINT (69.11915705764207 41.46696761534889),69.119157,41.466968,4310279974_7062_34325


In [23]:
test.shape, grid_graph.shape, grid.shape

((96033, 6), (96033, 3), (96033, 3))

In [26]:
osm_schools = get_amenities('Tashkent', 'schools')

/nfshome/gb1877/.conda/envs/osm2/lib/python3.6/site-packages/ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Saving OSM data to shapefile at path data/Tashkent


/nfshome/gb1877/.conda/envs/osm2/lib/python3.6/site-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [35]:
schools = gn.pandana_snap(G_drive, osm_schools)

In [38]:
schools_OD_drive = gn.calculate_OD(G=G_drive, origins=list(test.NN), destinations = list(schools.NN), 
                                 fail_value = 99999, weight='distance')

print origins type
<class 'list'>
weighted_origins equals false


In [42]:
schools_OD_drive[7632].min()

48.0

In [44]:
test['school_drive'] = [schools_OD_drive[i].min() for i in range(schools_OD_drive.shape[0])]

In [45]:
test.head()

,FID,centroid,geometry,x,y,NN,school_drive
0,0,POINT (69.11915705764207 41.47173977948322),POINT (69.11915705764207 41.47173977948322),69.119157,41.471740,4310279974_7053_34316,6.0
1,1,POINT (69.11915705764207 41.47054673844963),POINT (69.11915705764207 41.47054673844963),69.119157,41.470547,4310279974_7057_34320,2.0
2,2,POINT (69.11915705764207 41.46935369741605),POINT (69.11915705764207 41.46935369741605),69.119157,41.469354,4310279974_7059_34322,0.0
3,3,POINT (69.11915705764207 41.46816065638247),POINT (69.11915705764207 41.46816065638247),69.119157,41.468161,4310279974_7060_34323,1.0
4,4,POINT (69.11915705764207 41.46696761534889),POINT (69.11915705764207 41.46696761534889),69.119157,41.466968,4310279974_7062_34325,3.0


In [46]:
grid.head()

,FID,geometry,centroid
0,0,"POLYGON ((69.1187079 41.4723363, 69.1196062152...",POINT (69.11915705764207 41.47173977948322)
1,1,"POLYGON ((69.1187079 41.47114325896642, 69.119...",POINT (69.11915705764207 41.47054673844963)
2,2,"POLYGON ((69.1187079 41.46995021793284, 69.119...",POINT (69.11915705764207 41.46935369741605)
3,3,"POLYGON ((69.1187079 41.46875717689926, 69.119...",POINT (69.11915705764207 41.46816065638247)
4,4,"POLYGON ((69.1187079 41.46756413586568, 69.119...",POINT (69.11915705764207 41.46696761534889)


In [47]:
[min(schools_OD_drive[i]) for i in range(schools_OD_drive.shape[0])]

[6.0,
 2.0,
 0.0,
 1.0,
 3.0,
 4.0,
 6.0,
 7.0,
 8.0,
 9.0,
 10.0,
 10.0,
 10.0,
 10.0,
 4.0,
 2.0,
 1.0,
 0.0,
 0.0,
 1.0,
 3.0,
 6.0,
 6.0,
 6.0,
 7.0,
 8.0,
 8.0,
 10.0,
 10.0,
 10.0,
 10.0,
 10.0,
 11.0,
 11.0,
 11.0,
 12.0,
 13.0,
 13.0,
 14.0,
 14.0,
 14.0,
 14.0,
 16.0,
 16.0,
 16.0,
 17.0,
 18.0,
 18.0,
 18.0,
 20.0,
 20.0,
 20.0,
 20.0,
 20.0,
 21.0,
 21.0,
 22.0,
 23.0,
 23.0,
 23.0,
 23.0,
 23.0,
 24.0,
 25.0,
 26.0,
 26.0,
 27.0,
 27.0,
 27.0,
 28.0,
 29.0,
 30.0,
 31.0,
 31.0,
 31.0,
 33.0,
 35.0,
 35.0,
 36.0,
 36.0,
 36.0,
 36.0,
 37.0,
 37.0,
 38.0,
 40.0,
 40.0,
 41.0,
 43.0,
 43.0,
 43.0,
 43.0,
 44.0,
 45.0,
 47.0,
 46.0,
 50.0,
 52.0,
 52.0,
 53.0,
 53.0,
 56.0,
 58.0,
 109.0,
 111.0,
 112.0,
 112.0,
 111.0,
 109.0,
 113.0,
 116.0,
 114.0,
 111.0,
 109.0,
 106.0,
 102.0,
 99.0,
 96.0,
 100.0,
 103.0,
 106.0,
 109.0,
 111.0,
 126.0,
 126.0,
 126.0,
 133.0,
 133.0,
 133.0,
 133.0,
 133.0,
 131.0,
 131.0,
 131.0,
 131.0,
 131.0,
 131.0,
 131.0,
 130.0,
 130.0,
 130.0,
